In [1]:
import sys
sys.path.append('../')

import math
import gc
from timeit import default_timer as timer
from datetime import datetime, timedelta
import numba
import pandas as pd
import numpy as np

pd.options.display.max_rows=500
pd.options.display.max_columns=500

In [16]:
WHOLE_MARKET_COLUMNS = ['quoteDate', 'allordpreviousclose', 'allordchange', 'allorddayshigh', 'allorddayslow',
                        'allordpercebtChangeFrom52WeekHigh', 'allordpercentChangeFrom52WeekLow', 'asxpreviousclose',
                        'asxchange', 'asxdayshigh', 'asxdayslow', 'asxpercebtChangeFrom52WeekHigh',
                        'asxpercentChangeFrom52WeekLow', '640106_A3597525W', 'FIRMMCRT', 'FXRUSD', 'GRCPAIAD',
                        'GRCPAISAD', 'GRCPBCAD', 'GRCPBCSAD', 'GRCPBMAD', 'GRCPNRAD', 'GRCPRCAD', 'H01_GGDPCVGDP',
                        'H01_GGDPCVGDPFY', 'H05_GLFSEPTPOP']


In [48]:
df = pd.read_pickle('../data/ml-20190115-processed.pkl.gz', compression='gzip')
df.shape

(5384641, 265)

In [39]:
# df[df['volume']==0][['symbol', 'quoteDate']].head(100)

df[df['symbol']=='CG1'][['symbol', 'quoteDate', 'adjustedPrice', 'change', 'volume', 'FXRUSD', 'FXRUSD_T11_20P']].head(500)

,symbol,quoteDate,adjustedPrice,change,volume,FXRUSD,FXRUSD_T11_20P
quoteDate_ref,,,,,,,
2018-05-14,CG1,2018-05-14,0.430,NaN,27759,0.7458,NaN
2018-05-15,CG1,2018-05-15,0.425,-0.005,45633,0.7534,NaN
2018-05-16,CG1,2018-05-16,0.425,0.000,0,0.7508,NaN
2018-05-17,CG1,2018-05-17,0.420,-0.005,287372,0.7508,NaN
2018-05-18,CG1,2018-05-18,0.465,0.045,42373,0.7484,NaN
2018-05-21,CG1,2018-05-21,0.510,0.045,229346,0.7540,NaN
2018-05-22,CG1,2018-05-22,0.500,-0.010,2378,0.7519,NaN
2018-05-23,CG1,2018-05-23,0.490,-0.010,92622,0.7519,NaN
2018-05-24,CG1,2018-05-24,0.485,-0.005,2922,0.7588,NaN


In [43]:
df[['symbol', 'quoteDate']].groupby('symbol').count()

,quoteDate
symbol,
1ST,949
AAP,2955
ABV,2975
AMP,2978
AO1,1978
AOH,2761
AOU,1912
AR1,2613
AUC,2349


In [49]:
df_2 = pd.read_pickle('../data/ml-20190115-data.pkl.gz', compression='gzip')
df_2.shape

(3572726, 132)

In [47]:
df_2[['symbol', 'quoteDate']].groupby('symbol').count()

,quoteDate
symbol,
1ST,508
AAP,594
ABV,1775
AMP,2919
AO1,440
AOH,2602
AOU,948
AR1,1607
AUC,1691


In [45]:
df[df_2['symbol']=='CG1'][['symbol', 'quoteDate', 'adjustedPrice', 'change', 'volume', 'FXRUSD', 'FXRUSD_T11_20P']].head(500)

,symbol,quoteDate,adjustedPrice,change,volume,FXRUSD,FXRUSD_T11_20P
quoteDate_ref,,,,,,,
2018-05-14,CG1,2018-05-14,0.430,NaN,27759,0.7458,NaN
2018-05-15,CG1,2018-05-15,0.425,-0.005,45633,0.7534,NaN
2018-05-16,CG1,2018-05-16,0.425,0.000,0,0.7508,NaN
2018-05-17,CG1,2018-05-17,0.420,-0.005,287372,0.7508,NaN
2018-05-18,CG1,2018-05-18,0.465,0.045,42373,0.7484,NaN
2018-05-21,CG1,2018-05-21,0.510,0.045,229346,0.7540,NaN
2018-05-22,CG1,2018-05-22,0.500,-0.010,2378,0.7519,NaN
2018-05-23,CG1,2018-05-23,0.490,-0.010,92622,0.7519,NaN
2018-05-24,CG1,2018-05-24,0.485,-0.005,2922,0.7588,NaN


In [17]:
reference_whole_market_data = df_2[WHOLE_MARKET_COLUMNS].drop_duplicates()
reference_whole_market_data

,quoteDate,allordpreviousclose,allordchange,allorddayshigh,allorddayslow,allordpercebtChangeFrom52WeekHigh,allordpercentChangeFrom52WeekLow,asxpreviousclose,asxchange,asxdayshigh,asxdayslow,asxpercebtChangeFrom52WeekHigh,asxpercentChangeFrom52WeekLow,640106_A3597525W,FIRMMCRT,FXRUSD,GRCPAIAD,GRCPAISAD,GRCPBCAD,GRCPBCSAD,GRCPBMAD,GRCPNRAD,GRCPRCAD,H01_GGDPCVGDP,H01_GGDPCVGDPFY,H05_GLFSEPTPOP
quoteDate_ref,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-05-14,2018-05-14,6216.399902,18.600098,6235.000000,6211.799805,-0.006409,0.096232,6116.200195,19.099609,6135.500000,6109.299805,-0.005496,0.086397,112.599998,1.500000,0.7458,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.923405
2018-05-15,2018-05-15,6235.000000,-36.299805,6245.899902,6198.700195,-0.003436,0.099512,6135.299805,-37.500000,6146.799805,6097.799805,-0.002390,0.089790,112.599998,1.500000,0.7534,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.923405
2018-05-16,2018-05-16,6235.000000,-36.299805,6245.899902,6198.700195,-0.003436,0.099512,6135.299805,-37.500000,6146.799805,6097.799805,-0.002390,0.089790,112.599998,1.500000,0.7534,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.923405
2018-05-17,2018-05-17,6208.100098,-10.899902,6220.500000,6181.899902,-0.007736,0.094768,6107.000000,-12.700195,6120.200195,6077.600098,-0.006992,0.084763,112.599998,1.500000,0.7508,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.923405
2018-05-18,2018-05-18,6197.200195,-6.300293,6212.700195,6183.399902,-0.009478,0.092846,6094.299805,-6.899902,6110.600098,6079.500000,-0.009057,0.082507,112.599998,1.500000,0.7484,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.923405
2018-05-21,2018-05-21,6190.899902,-0.699707,6200.799805,6176.200195,-0.010485,0.091735,6087.399902,-2.899902,6094.899902,6070.299805,-0.010179,0.081282,112.599998,1.500000,0.7540,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.934814
2018-05-22,2018-05-22,6190.200195,-40.300293,6190.200195,6138.299805,-0.010597,0.091611,6084.500000,-42.600098,6084.500000,6028.500000,-0.010650,0.080767,112.599998,1.500000,0.7519,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.934814
2018-05-23,2018-05-23,6149.899902,-9.600098,6163.899902,6135.299805,-0.017038,0.084505,6041.899902,-9.399902,6056.600098,6026.100098,-0.017577,0.073200,112.599998,1.500000,0.7519,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.934814
2018-05-24,2018-05-24,6140.299805,3.800293,6147.899902,6120.799805,-0.018573,0.082812,6032.500000,4.600098,6039.299805,6011.899902,-0.019106,0.071530,112.599998,1.500000,0.7588,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.934814


In [18]:
reference_whole_market_data.index.duplicated()

array([False, False, False, ...,  True,  True,  True])

In [25]:
reference_whole_market_data.shape

(42166, 26)

In [28]:
reference_whole_market_data[reference_whole_market_data.index.duplicated()]

,quoteDate,allordpreviousclose,allordchange,allorddayshigh,allorddayslow,allordpercebtChangeFrom52WeekHigh,allordpercentChangeFrom52WeekLow,asxpreviousclose,asxchange,asxdayshigh,asxdayslow,asxpercebtChangeFrom52WeekHigh,asxpercentChangeFrom52WeekLow,640106_A3597525W,FIRMMCRT,FXRUSD,GRCPAIAD,GRCPAISAD,GRCPBCAD,GRCPBCSAD,GRCPBMAD,GRCPNRAD,GRCPRCAD,H01_GGDPCVGDP,H01_GGDPCVGDPFY,H05_GLFSEPTPOP
quoteDate_ref,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-05-16,2018-05-16,6208.100098,-10.899902,6220.500000,6181.899902,-0.007736,0.094768,6107.000000,-12.700195,6120.200195,6077.600098,-0.006992,0.084763,112.599998,1.500000,0.7508,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.923405
2018-05-22,2018-05-22,6190.899902,-0.699707,6200.799805,6176.200195,-0.010485,0.091735,6087.399902,-2.899902,6094.899902,6070.299805,-0.010179,0.081282,112.599998,1.500000,0.7540,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.934814
2018-05-28,2018-05-28,6144.100098,-3.100098,6145.200195,6123.600098,-0.017965,0.083482,6037.100098,-4.300293,6038.899902,6016.600098,-0.018358,0.072347,112.599998,1.500000,0.7541,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.934814
2018-05-29,2018-05-29,6144.100098,-3.100098,6145.200195,6123.600098,-0.017965,0.083482,6037.100098,-4.300293,6038.899902,6016.600098,-0.018358,0.072347,112.599998,1.500000,0.7541,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.934814
2018-05-30,2018-05-30,6144.100098,-3.100098,6145.200195,6123.600098,-0.017965,0.083482,6037.100098,-4.300293,6038.899902,6016.600098,-0.018358,0.072347,112.599998,1.500000,0.7541,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.934814
2018-05-31,2018-05-31,6144.100098,-3.100098,6145.200195,6123.600098,-0.017965,0.083482,6037.100098,-4.300293,6038.899902,6016.600098,-0.018358,0.072347,112.599998,1.500000,0.7541,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.934814
2018-06-01,2018-06-01,6144.100098,-3.100098,6145.200195,6123.600098,-0.017965,0.083482,6037.100098,-4.300293,6038.899902,6016.600098,-0.018358,0.072347,112.599998,1.500000,0.7541,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.934814
2018-06-04,2018-06-04,6144.100098,-3.100098,6145.200195,6123.600098,-0.017965,0.083482,6037.100098,-4.300293,6038.899902,6016.600098,-0.018358,0.072347,112.599998,1.500000,0.7541,109.012474,105.716202,101.879692,96.319092,122.190910,109.332916,106.764053,432685.0,0.796353,61.934814
2018-06-12,2018-06-12,6169.399902,-12.600098,6172.799805,6153.200195,-0.013922,0.084539,6057.299805,-12.099609,6060.799805,6041.399902,-0.015073,0.074104,112.599998,1.500000,0.7664,109.294136,107.062073,102.443825,98.738426,125.209503,109.163177,110.213043,432685.0,0.796353,61.934814


In [37]:
rdf = reference_whole_market_data.groupby('quoteDate').first()

In [30]:
reference_whole_market_data.groupby('quoteDate').first().shape

(2978, 25)

In [35]:
rdf['quoteDate']

0      2007-07-02
1      2007-07-03
2      2007-07-04
3      2007-07-05
4      2007-07-06
5      2007-07-09
6      2007-07-10
7      2007-07-11
8      2007-07-12
9      2007-07-13
10     2007-07-16
11     2007-07-17
12     2007-07-18
13     2007-07-19
14     2007-07-20
15     2007-07-23
16     2007-07-24
17     2007-07-25
18     2007-07-26
19     2007-07-27
20     2007-07-30
21     2007-07-31
22     2007-08-01
23     2007-08-02
24     2007-08-03
25     2007-08-06
26     2007-08-07
27     2007-08-08
28     2007-08-09
29     2007-08-10
30     2007-08-13
31     2007-08-14
32     2007-08-15
33     2007-08-16
34     2007-08-17
35     2007-08-20
36     2007-08-21
37     2007-08-22
38     2007-08-23
39     2007-08-24
40     2007-08-27
41     2007-08-28
42     2007-08-29
43     2007-08-30
44     2007-08-31
45     2007-09-03
46     2007-09-04
47     2007-09-05
48     2007-09-06
49     2007-09-07
50     2007-09-10
51     2007-09-11
52     2007-09-12
53     2007-09-13
54     2007-09-14
55     200

In [36]:
rdf.index

RangeIndex(start=0, stop=2978, step=1)